In [ ]:
import pandas as pd
import os
import numpy as np

In [ ]:
# 改成你放代码的路径
ROOT_PATH = '/content/drive/MyDrive/MSBD5001-Final/sp500'

os.chdir(ROOT_PATH)
os.listdir(ROOT_PATH)

# 读取sp500folder中存在的所有文件夹
dirs = os.listdir()

for stock in dirs:
  # read data
  path = os.path.join(ROOT_PATH, stock)
  files = os.listdir(path)
  
  if stock+'_OHLC_1d.csv' not in files or stock+'_info.csv' not in files:
    # 如果没有OHLC文件，说明股票不存在，跳过
    continue

  df = pd.read_csv(path+'/'+stock+'_OHLC_1d.csv')
  info = pd.read_csv(path+'/'+stock+'_info.csv')  # info

  # drop rows with 'NAN' in month data
  df = df.dropna(axis=0,how='any')
  df = df.drop(columns=['Date'])
  # df['Stock'] = stock
  break



下面的代码基于上面获取数据集得到的dataframe，每次训练并预测得到一支股票的结果（基于该股票的dataframe）。可以根据需求进行修改。（但是别改git仓库里的）

##LSTM

写代码的时候懒得切汉语輸入法写注释了就用英文写的，有不懂的地方记得问我
下面的代码是分切了测试集训练集的，实际做quant因子的时候，无需分割，把下面training_data_len直接就是len(dataset)即可.
深度模型的多步预测尽量使用多个模型分别预测。
get_train(1,train_data)处可以多得到适应于每一个steps的测试数据，其中第一个参数为后续要预测的steps数。
同时，每增加一个step（默认为3），则需要增加一个model，在train部分中可以添加新的model，已经使用了工厂设计模式，直接得到新的model并投入新的测试数据即可进行训练。
prediction部分示例了训练好后的模型如何使用，你们可以根据需要进行使用。    
示例使用如下。

### data prepare

In [ ]:
# Convert the dataframe to a numpy array
# df this the dataframe on one stock
dataset = df.values
# Get the number of rows to train the model on
training_data_len = int(np.ceil( len(dataset) * .95 ))

# Scale the data
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)

scaled_data[0,1]

# Create the training data set 
# Create the scaled training data set
train_data = scaled_data[0:int(training_data_len), :]

# Split the data into x_train and y_train data sets
# get more data for multisteps prediction
def get_train(steps,train_data):
    x_train = []
    y_train = []

    for i in range(60, len(train_data)-steps+1):
        x_train.append(train_data[i-60:i])
        y_train.append(train_data[i+steps-1])
        if i<= 61:
            print(x_train)
            print(y_train)
            print()

    # Convert the x_train and y_train to numpy arrays 
    x_train, y_train = np.array(x_train), np.array(y_train)

    # Reshape the data
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], x_train.shape[2]))
    return x_train, y_train
# x_train_1 means train data for step ==1
x_train_1,y_train_1 = get_train(1,train_data)
x_train_2,y_train_2 = get_train(2,train_data)
x_train_3,y_train_3 = get_train(3,train_data)






###train

In [ ]:

import tensorflow as tf
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

from keras.models import Sequential
from keras.layers import Dense, LSTM

# Train
# Build the LSTM model
def model_factory(x_train,y_train):
    model = Sequential()
    model.add(LSTM(128, return_sequences=True, input_shape= (x_train.shape[1], x_train.shape[2])))
    model.add(LSTM(64, return_sequences=False))
    model.add(Dense(25))
    model.add(Dense(y_train.shape[1]))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model


model_1 = model_factory(x_train_1,y_train_1)
model_2 = model_factory(x_train_2,y_train_2)
model_3 = model_factory(x_train_3,y_train_3)

# Train the model
model_1.fit(x_train_1, y_train_1, batch_size=5, epochs=15)
model_2.fit(x_train_2, y_train_2, batch_size=5, epochs=15)
model_3.fit(x_train_3, y_train_3, batch_size=5, epochs=15)

###prediction

In [10]:
# use last data to predict next 3 steps
test_data = scaled_data[training_data_len - 60: , :]
x_test = []
y_test = dataset[training_data_len:, :]
for i in range(60, 61):
    x_test.append(test_data[i-60:i])
    
# Convert the data to a numpy array
x_test_np = np.array(x_test)

# Reshape the data
x_test_np = np.reshape(x_test_np, (x_test_np.shape[0], x_test_np.shape[1], x_test_np.shape[2] ))
predictions1 = model_1.predict(x_test_np)
predictions1 = scaler.inverse_transform(predictions1)
predictions2 = model_2.predict(x_test_np)
predictions2 = scaler.inverse_transform(predictions2)
predictions3 = model_3.predict(x_test_np)
predictions3 = scaler.inverse_transform(predictions3)
print(predictions1[0])
print(y_test[0])
print("------------------------------")
print(predictions2[0])
print(y_test[1])
print("------------------------------")
print(predictions3[0])
print(y_test[2])

1/1 [==============================] - 1s 647ms/step
[3.5947119e+02 3.6796805e+02 3.5189447e+02 3.6208990e+02 5.5244855e+06]
[3.55019989e+02 3.72799988e+02 3.53510010e+02 3.62989990e+02
 5.34350000e+06]
------------------------------
[3.6342960e+02 3.6876837e+02 3.5335291e+02 3.6150275e+02 5.2016930e+06]
[3.60790009e+02 3.72549988e+02 3.58220001e+02 3.65329987e+02
 3.16820000e+06]
------------------------------
[3.5972644e+02 3.6505960e+02 3.4779260e+02 3.5513913e+02 4.8447050e+06]
[3.71709991e+02 3.78179993e+02 3.67470001e+02 3.76640015e+02
 2.53430000e+06]


##ARIMA

在这个部分我直接定义了一个class 方便你们调用使用,做了slice操作只是为了测试训练效果，实际使用时应该直接使用  **df['Close']**  整个丢进去。
示例使用如下。

In [26]:

from statsmodels.tsa.arima.model import ARIMA


class ARIMA_class:
    def __init__(self, train_x, time_window=5):
        # input args train_x-list
        self.model = None
        self.model_fit = None
        self.train_x = train_x
        self.time_window = time_window
        self.arima_pred = []

    def fit(self):
        # 阶数等于windwos大小，即与历史前多少天数据相关
        self.model = ARIMA(self.train_x, order=(self.time_window, 1, self.time_window)) 
        self.model_fit = self.model.fit()

    def predict(self, valid_len):
        # return type pandas.dataframe
        forecast = self.model_fit.forecast(valid_len)  
        # forecast = forecast.reset_index(drop=True)
        return forecast


In [27]:
type(df['Close'])

pandas.core.series.Series

In [28]:
arima_model = ARIMA_class(df['Close'][:-3])

In [29]:
arima_model.fit()

/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)


In [30]:
arima_model.predict(3)


1444    273.864872
1445    274.747956
1446    273.744325
Name: predicted_mean, dtype: float64

In [31]:
df['Close'][-3:]

1444    281.399994
1445    278.250000
1446    275.200012
Name: Close, dtype: float64

##VAR

该部分默认做了一阶差分.
同时，该模型严重依赖于过去时间的股价，导致结果严重受影响于当前股价，私以为可以用该模型预测涨跌走势但是不太适合预测价格本身。
var_class中的predict函数的第一个参数为用于预测的数据集，一般用倒数var_model.model.k_ar个，第二个参数为要预测的天数。
示例使用如下。

In [53]:

from statsmodels.tsa.vector_ar.var_model import VAR
class VAR_class:
    def __init__(self, train_x, time_window=5):
        self.train_x_diff = None
        self.model = None
        self.train_x = train_x
        self.time_window = time_window
    def diff(self):
        self.train_x_diff = self.train_x.diff()
        self.train_x_diff = self.train_x_diff.fillna(method="bfill")

    def inverse_diff(self,var_pred, second_diff=False):
        df_fc = var_pred.copy()
        columns = self.train_x.columns
        for col in columns:
            # # Roll back 2nd Diff
            # if second_diff:
            #     df_fc[str(col)+'_1d'] = (self.train_x[col].iloc[-1]-self.train_x[col].iloc[-2]) + df_fc[str(col)+'_2d'].cumsum()
            df_fc[str(col) + '_pred'] = self.train_x[col].iloc[0] + df_fc[str(col) + '_pred'].cumsum()
        var_pred = df_fc
        return var_pred

    def fit(self,diff_times=1):
        for i in range(diff_times):
            self.diff()
        self.model = VAR(endog= self.train_x_diff)
        self.model = self.model.fit(maxlags=self.time_window)

    def predict(self,test_x,valid_len):
        test_x = test_x.diff()
        test_x = test_x.fillna(method="bfill")
        var_pred = self.model.forecast(test_x.values, steps=valid_len)
        var_pred = pd.DataFrame(var_pred, columns=self.train_x.columns + '_pred')
        var_pred = self.inverse_diff(var_pred)
        return  var_pred['Close_pred']

In [54]:
var_model = VAR_class(df[:-3])


In [55]:
var_model.fit()

In [56]:
lag_order = var_model.model.k_ar
var_model.predict(df[-lag_order:],3)

0    102.329694
1    102.770830
2    104.314593
Name: Close_pred, dtype: float64

In [57]:
df[-3:]['Close']

1444    281.399994
1445    278.250000
1446    275.200012
Name: Close, dtype: float64